In [121]:
import pandas as pd
import numpy as np


In [122]:
df= pd.read_csv('eplmatches.csv')
df

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A
...,...,...,...,...,...,...,...,...
11641,2022,9,2021-10-23,Leeds United,1,1,Wolves,D
11642,2022,9,2021-10-23,Brighton,1,4,Manchester City,A
11643,2022,9,2021-10-24,West Ham,1,0,Tottenham,H
11644,2022,9,2021-10-24,Brentford,1,2,Leicester City,A


In [148]:
df.Home.unique()

array(['Coventry City', 'Leeds United', 'Sheffield Utd', 'Crystal Palace',
       'Arsenal', 'Ipswich Town', 'Everton', 'Southampton', 'Chelsea',
       "Nott'ham Forest", 'Manchester City', 'Blackburn',
       'Manchester Utd', 'Liverpool', 'Middlesbrough', 'Sheffield Weds',
       'Wimbledon', 'QPR', 'Oldham Athletic', 'Tottenham', 'Norwich City',
       'Aston Villa', 'Newcastle Utd', 'West Ham', 'Swindon Town',
       'Leicester City', 'Bolton', 'Sunderland', 'Derby County',
       'Barnsley', 'Charlton Ath', 'Watford', 'Bradford City', 'Fulham',
       'West Brom', 'Birmingham City', 'Portsmouth', 'Wolves',
       'Wigan Athletic', 'Reading', 'Hull City', 'Stoke City', 'Burnley',
       'Blackpool', 'Swansea City', 'Cardiff City', 'Bournemouth',
       'Brighton', 'Huddersfield', 'Brentford'], dtype=object)

In [124]:
df.loc[df['FTR'] =='H', 'Won'] = df['Home']
df.loc[df['FTR'] =='A', 'Won'] = df['Away']
df.loc[df['FTR'] =='H', 'Lost'] =df['Away']
df.loc[df['FTR'] =='A', 'Lost'] =df['Home']
df.loc[df['FTR'] =='D', 'HomeDraw'] = df['Home']
df.loc[df['FTR'] =='D', 'AwayDraw'] = df['Away']

In [125]:
df=df[df.Wk<20] # Getting only the games till the mid-season
"""with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)"""
df

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR,Won,Lost,HomeDraw,AwayDraw
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H,Coventry City,Middlesbrough,NaN,NaN
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H,Leeds United,Wimbledon,NaN,NaN
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H,Sheffield Utd,Manchester Utd,NaN,NaN
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D,NaN,NaN,Crystal Palace,Blackburn
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A,Norwich City,Arsenal,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
11641,2022,9,2021-10-23,Leeds United,1,1,Wolves,D,NaN,NaN,Leeds United,Wolves
11642,2022,9,2021-10-23,Brighton,1,4,Manchester City,A,Manchester City,Brighton,NaN,NaN
11643,2022,9,2021-10-24,West Ham,1,0,Tottenham,H,West Ham,Tottenham,NaN,NaN
11644,2022,9,2021-10-24,Brentford,1,2,Leicester City,A,Leicester City,Brentford,NaN,NaN


In [126]:
CountWin = df.groupby(['Season_End_Year', 'Won']).agg({'Won': ['count']})
CountWin.columns = ['CountWins']
CountWin=CountWin.reset_index()


In [151]:
CountWin
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(CountWin)

     Season_End_Year              Won  CountWins
0               1993          Arsenal          9
1               1993      Aston Villa          9
2               1993        Blackburn          8
3               1993          Chelsea          9
4               1993    Coventry City          6
5               1993   Crystal Palace          3
6               1993          Everton          5
7               1993     Ipswich Town          6
8               1993     Leeds United          6
9               1993        Liverpool          8
10              1993  Manchester City          7
11              1993   Manchester Utd          9
12              1993    Middlesbrough          6
13              1993     Norwich City         12
14              1993  Nott'ham Forest          3
15              1993  Oldham Athletic          4
16              1993              QPR          8
17              1993    Sheffield Utd          5
18              1993   Sheffield Weds          4
19              1993

In [154]:
Relegated=pd.DataFrame({'Season_End_Year':[1993,1993,1993],'Won':['Crystal Palace','Middlesbrough','Nott\'ham Forest']})
Relegated

,Season_End_Year,Won
0,1993,Crystal Palace
1,1993,Middlesbrough
2,1993,Nott'ham Forest


In [155]:
Relegated=pd.merge(CountWin, Relegated, on=['Season_End_Year', 'Won'])
Relegated

,Season_End_Year,Won,CountWins
0,1993,Crystal Palace,3
1,1993,Middlesbrough,6
2,1993,Nott'ham Forest,3
